In [1]:
from keras.models import Sequential
import tensorflow as tf
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, Reshape, Activation
from keras.datasets import cifar10
from sklearn.metrics import log_loss

In [2]:
def vgg8_model(img_rows, img_cols, channel):
   
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(img_rows, img_cols, channel)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    # Add Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(3072, activation='relu'))
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(2048, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    # Learning rate is changed to 0.001
    sgd = SGD(learning_rate=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    # adm = Adam(learning_rate=0.001) 
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [3]:
import cv2
import numpy as np

from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

nb_train_samples = 10000 # 5000 training samples
nb_valid_samples = 1000 # 300 validation samples
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

    # Resize trainging images
    if K.image_data_format() == 'th':
        X_train = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img.transpose(1,2,0), (img_rows,img_cols)).transpose(2,0,1) for img in X_valid[:nb_valid_samples,:,:,:]])
    else:
        X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
        X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
    Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid

In [4]:
if __name__ == '__main__':

    # Example to fine-tune on 5000 samples from Cifar10

    img_rows, img_cols = 224, 224 # Resolution of inputs
    channel = 3 
    batch_size = 100 
    nb_epoch = 30

    

    # Load Cifar10 data. Please implement your own load_data() module for your own dataset
    X_train, Y_train, X_valid, Y_valid = load_cifar10_data(img_rows, img_cols)
   
    

    # Load vgg7 our model
    model = vgg8_model(img_rows, img_cols, channel)

    import tensorflow as tf

    # Runnin on GPU
    with tf.device('/gpu:0'):
        # Start Fine-tuning
        print('Using GPU')
        model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              shuffle=True,
              verbose=1,
              validation_data=(X_valid, Y_valid),
              )

    # Make predictions
    predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

    # Cross-entropy loss score
    score = log_loss(Y_valid, predictions_valid)

Using GPU
Epoch 1/30
100/100 - 65s - loss: 2.2693 - accuracy: 0.1343 - val_loss: 2.1064 - val_accuracy: 0.2310 - 65s/epoch - 653ms/step
Epoch 2/30
100/100 - 56s - loss: 1.9541 - accuracy: 0.2872 - val_loss: 1.8239 - val_accuracy: 0.3530 - 56s/epoch - 563ms/step
Epoch 3/30
100/100 - 54s - loss: 1.7534 - accuracy: 0.3715 - val_loss: 1.7066 - val_accuracy: 0.3760 - 54s/epoch - 543ms/step
Epoch 4/30
100/100 - 60s - loss: 1.6393 - accuracy: 0.4112 - val_loss: 1.6462 - val_accuracy: 0.4090 - 60s/epoch - 595ms/step
Epoch 5/30
100/100 - 54s - loss: 1.5508 - accuracy: 0.4485 - val_loss: 1.6230 - val_accuracy: 0.4080 - 54s/epoch - 539ms/step
Epoch 6/30
100/100 - 61s - loss: 1.4860 - accuracy: 0.4678 - val_loss: 1.5780 - val_accuracy: 0.4380 - 61s/epoch - 609ms/step
Epoch 7/30
100/100 - 59s - loss: 1.4226 - accuracy: 0.4942 - val_loss: 1.5147 - val_accuracy: 0.4650 - 59s/epoch - 587ms/step
Epoch 8/30
100/100 - 55s - loss: 1.3441 - accuracy: 0.5183 - val_loss: 1.5321 - val_accuracy: 0.4360 - 55s/e